# Import Library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
from copy import deepcopy

# Classes and Function

In [ ]:
class Point: 
    def __init__(self, id, x, y, partition):
        self.id = id
        self.partition = partition
        self.x = x
        self.y = y 

    def get_pos(self):
        return self.x, self.y

In [ ]:
class Graph: 
    def __init__(self, points = {}, weight = np.array([[]])): 
        self.points = points
        self.weight = weight
        self.all_weight = weight

    def get_points(self):
        return self.points        
        
    def get_partition_points(self, partition):
        temp = {}
        for i in self.points: 
            if self.points[i].get('partition') == partition: 
                temp[i] = self.points[i]
                
        return temp

    def get_partition(self):
        partition_result = np.array([])
        for i in self.points:
            partition_result = np.append(partition_result, self.points[i].get('partition'))

        return partition_result

        
    def get_points_coords(self): 
        points_x = np.array([])
        points_y = np.array([])

        for i in self.points:
            points_x = np.append(points_x, self.points[i].get('x'))
            points_y = np.append(points_y, self.points[i].get('y'))

        return points_x, points_y
    
    def get_partition_points_coords(self, partition):
        points_x = np.array([])
        points_y = np.array([])

        for i in self.get_partition_points(partition):
            points_x = np.append(points_x, self.points[i].get('x'))
            points_y = np.append(points_y, self.points[i].get('y'))

        return points_x, points_y

    def get_weight(self):
        return self.weight

    def get_spread(self):
        points_x, points_y = self.get_points_coords()
        return max(max(points_x) - min(points_y), max(points_y) - min(points_y))

    def get_partition_spread(self, partition):
        points_x, points_y = self.get_partition_points_coords(partition)
        if(len(points_x) == 0):
            return 0
        return max(max(points_x) - min(points_y), max(points_y) - min(points_y))

    def get_edge_cut(self):
        edge_cut = {}
        size = len(self.points)
        for i in range(size):
            for j in range(i, size):
                if(weight[j, i] != 0 and self.points[i].get('partition') != self.points[j].get('partition')):
                    edge_cut[(j, i)] = weight[j, i]
        return edge_cut

    def get_partition_size(self, partition):
        points_x, _ = self.get_partition_points_coords(partition)
        return len(points_x)

    def is_connected(self):
        if(self.weight.size == 0): 
            return False
            
        num_vertices = len(self.points)

        def dfs(vertex, visited):
            visited[vertex] = True
            for neighbor in range(num_vertices):
                if self.weight[vertex, neighbor] != 0 and not visited[neighbor]:
                    dfs(neighbor, visited)

        visited = [False] * num_vertices

        dfs(0, visited)

        return all(visited) 

    def is_partition_connected(self, partition):
        if(self.weight.size == 0): 
            return False
        
        num_vertices = len(self.get_partition_points(partition))

        def dfs(vertex, visited, idx=-1):
            visited[idx] = True
            for i, neighbor in enumerate(self.get_partition_points(partition)):
                if self.weight[vertex, neighbor] != 0 and not visited[i]:
                    dfs(neighbor, visited, i)

        visited = [False] * num_vertices
        dfs([v for v in self.get_partition_points(partition)][0], visited, 0)

        return all(visited)          

    def calculate_weight(self, min_s, max_s):
        iteration = 0
        points_x, points_y = self.get_points_coords()
        size = len(points_x)
        temp = np.zeros([size, size])
        for i in range(size):
            for j in range(size):
                temp[i, j] = np.sqrt(np.abs(points_x[i] - points_x[j]) ** 2 + np.abs(points_y[i] - points_y[j]) ** 2)
        
        self.all_weight = np.copy(temp)
        self.weight = np.zeros([size, size])
        while(not self.is_connected()):
            iteration += 1
            if(min_s != 0 and max_s != 0):
                new_weight = np.zeros([size, size])
                
                for j in range(size):
                    n_edge_rand = np.random.randint(min_s, max_s)
                    idx = np.argpartition(self.all_weight[j, :], n_edge_rand)
                    for k in idx[1:n_edge_rand + 1]:
                        new_weight[j, k] = self.all_weight[j, k]
                    new_weight[:, j] = new_weight[j, :]
                    
            self.weight = np.copy(new_weight)

            if(iteration % 100 == 0):
                if(min_s < max_s - 1):
                    min_s += 1
                else:
                    max_s += 1
                    
        print('Iterations: ', iteration, 'Min Side:', min_s, 'Max Side:', max_s)
        return self.weight

    def update_partition(self, id, partition):
        self.points[id]['partition'] = partition

    def add_point(self, point): 
        self.points[point.id] = {'x': point.x, 'y': point.y, 'partition': point.partition}

    def remove_point(self, id):
        try:
            self.points.pop(id)
        except:
            print('point not in partition')

    def mutate_graph(self):
        G = nx.Graph(self.get_weight())
        while True:
            random = np.random.randint(0, len(weight))
            neighbor = [n for n in G.neighbors(random)]
            random_neighbor = np.random.randint(0, len(neighbor))

            if(self.points[neighbor[random_neighbor]].get("partition") != self.points[random].get("partition")):
                prev_partition = self.points[random].get("partition")
                self.update_partition(random, self.points[(neighbor[random_neighbor])].get("partition"))

                if(self.is_partition_connected(prev_partition)):
                    break
                else: 
                    self.update_partition(random, prev_partition)

In [ ]:
def generate_rand_points(graph, n, interval): 
    temp = np.random.rand(2, n) * interval[0] + interval[1]
    for i in range(n):
        graph.add_point(Point(i, temp[0, i], temp[1, i], -1))
    return graph

## Generate Graph

In [ ]:
n_points = 100
n_partition = 5
interval = [-30, 30]

graph = generate_rand_points(Graph(), n_points, interval)
points_x, points_y = graph.get_points_coords()

In [ ]:
weight = graph.calculate_weight(1, 2)
pos = {i: (graph.get_points()[i].get("x"), graph.get_points()[i].get("y")) for i in graph.get_points()}

G = nx.Graph(np.array(weight))
nx.draw(G, node_size=50, pos=pos)
plt.axis("on")
plt.show()

In [ ]:
def random_connected_partitions(graph, adjacency_matrix, num_partitions):
    G = nx.Graph(np.array(adjacency_matrix))
    G_full = nx.Graph(np.array(adjacency_matrix))

    def bfs_limit(graph, source, min_size):
        visited = set()
        queue = [source]

        while queue and len(visited) < min_size:
            node = queue.pop(0)
            if node not in visited:
                visited.add(node)
                queue.extend(neighbor for neighbor in graph.neighbors(node) if neighbor not in visited)

        return visited
        
    nodes = list(G.nodes())

    partition_size = len(nodes) // num_partitions // 2
    partitions = []
    current_degree = 1

    for i in range(num_partitions):
        degree = {}
        for j in G.nodes():
            temp = degree.get(G.degree(j), np.array([]))
            temp = np.append(temp, int(j))
            np.random.shuffle(temp)
            degree[G.degree(j)] = temp
            
        while True:
            if(len(degree.get(current_degree, [])) == 0):
                current_degree += 1
            else: 
                temp = degree.get(current_degree)
                np.random.shuffle(temp)

                start_node = degree.get(current_degree)[0]
                break

        partition = list(bfs_limit(G, start_node, partition_size))
        G.remove_nodes_from(partition)
        partitions.append(partition)
        
    while (len(G.nodes())):
        updated_node = []
        for i in G.nodes():
            index, min_partition = -1, -1
            for j, val in enumerate([n for n in G_full.neighbors(i)]):
                for k, val_p in enumerate(partitions): 
                    if(val in val_p and min_partition == -1):
                        index = k
                        min_partition = len(val_p)
                    elif(val in val_p and len(val_p) < min_partition):
                        index = k
                        min_partition = len(val_p)
            if(min_partition != -1):
                updated_node.append(i)
                partitions[index].append(i)
        G.remove_nodes_from(updated_node)

    partition_vector = np.zeros(len(nodes), dtype=int)
    for i, partition in enumerate(partitions):
        partition_vector[[int(j) for j in partition]] = i + 1

    for i, val in enumerate(partition_vector):
        graph.update_partition(i, val)


In [ ]:
np.random.seed(8)
random_connected_partitions(graph, weight, n_partition)
partition_result = graph.get_partition()
node_colors = [partition_result[node] for node in G.nodes()]
labels = {node: f'{node}' for node in G.nodes()}
label_pos = {k: (v[0], v[1]) for k, v in pos.items()} 

nx.draw(G, pos, node_size=100, node_color=node_colors, cmap=plt.cm.tab20b)
nx.draw_networkx_labels(G, pos=label_pos, labels=labels, font_size=8, font_color='whitesmoke', font_weight='800')

plt.show()

# Optimization 

## Objective Function

In [ ]:
def f1_obj(edge_cut):
    sum = 0
    for i in edge_cut: 
        sum += edge_cut.get(i, 0)
    return sum

def f2_obj(graph, n):
    sum = 0
    for i in range(n - 1):
        for j in range(i, n):
            sum += np.abs(graph.get_partition_size(i + 1) - graph.get_partition_size(j + 1)) 
    return sum

def f3_obj(graph, n):
    sum = 0
    for i in range(n):
        sum += graph.get_partition_spread(i + 1)
    return sum

## Selection 

In [ ]:
n_population = 3
populations = np.array([])
partition_results = np.array([])

for i in tqdm(range(n_population)):
    new_graph = deepcopy(graph) 
    partition_result = random_connected_partitions(new_graph, weight, n_partition)
    populations = np.append(new_graph, populations)
    partition_results = np.append(partition_results, partition_result)

In [ ]:
for population in populations:
    partition_result = population.get_partition()
    node_colors = [partition_result[node] for node in G.nodes()]
    labels = {node: f'{node}' for node in G.nodes()}
    # labels = {node: f'{int(partition_result[node])}' for node in G.nodes()}
    label_pos = {k: (v[0], v[1]) for k, v in pos.items()}  

    plt.figure()
    nx.draw(G, pos, node_size=100, node_color=node_colors, cmap=plt.cm.tab20b)
    nx.draw_networkx_labels(G, pos=label_pos, labels=labels, font_size=8, font_color='white', font_weight='800')

    plt.show()

In [ ]:
for i, population in enumerate(populations):
    print(f"Population-{i + 1}")
    print(f"> f1 = {f1_obj(populations[i].get_edge_cut())}")
    print(f"> f2 = {f2_obj(population, n_partition)}")
    print(f"> f3 = {f3_obj(population, n_partition)}")


## Mutation

In [ ]:
for population in populations[:2]:
    population.mutate_graph()
    partition_result = population.get_partition()
    node_colors = [partition_result[node] for node in G.nodes()]
    labels = {node: f'{node}' for node in G.nodes()}
    label_pos = {k: (v[0], v[1]) for k, v in pos.items()}  

    plt.figure()
    nx.draw(G, pos, node_size=100, node_color=node_colors, cmap=plt.cm.tab20b)
    nx.draw_networkx_labels(G, pos=label_pos, labels=labels, font_size=8, font_color='white', font_weight='800')

    plt.show()

In [ ]:
for i, population in enumerate(populations):
    print(f"Population-{i + 1}")
    print(f"> f1 = {f1_obj(populations[i].get_edge_cut())}")
    print(f"> f2 = {f2_obj(population, n_partition)}")
    print(f"> f3 = {f3_obj(population, n_partition)}")


## Crossover

In [ ]:
print(populations[0].get_partition_points(4))
print(populations[1].get_partition_points(2))

In [ ]:
population_a = populations[0]
population_b = populations[1]
population_a_target = 4
population_b_target = 2

temp_a_cross = {}
temp_b_cross = {}

population_a_temp = deepcopy(population_a)
population_b_temp = deepcopy(population_b)

for i in population_a_temp.get_partition_points(population_a_target):
    partition = population_b.get_points().get(i).get('partition')
    val = temp_a_cross.get(partition, 0)
    temp_a_cross[partition] = val + 1

for i in population_b.get_partition_points(max(temp_a_cross, key=temp_a_cross.get)):
    population_b.update_partition(i, 0)
    
for i in population_a_temp.get_partition_points(population_a_target):
    population_b.update_partition(i, max(temp_a_cross, key=temp_a_cross.get))

for i in population_b_temp.get_partition_points(population_b_target):
    partition = population_a.get_points().get(i).get('partition')
    val = temp_b_cross.get(partition, 0)
    temp_b_cross[partition] = val + 1

for i in population_a.get_partition_points(max(temp_b_cross, key=temp_b_cross.get)):
    population_a.update_partition(i, 0)
    
for i in population_b_temp.get_partition_points(population_b_target):
    population_a.update_partition(i, max(temp_a_cross, key=temp_a_cross.get))


In [ ]:
population_a.get_partition_points(0)

In [ ]:
population_b.get_partition_points(0)

In [ ]:
for population in populations[:2]:
    population.mutate_graph()
    partition_result = population.get_partition()
    node_colors = [partition_result[node] for node in G.nodes()]
    labels = {node: f'{node}' for node in G.nodes()}
    label_pos = {k: (v[0], v[1]) for k, v in pos.items()}  

    plt.figure()
    nx.draw(G, pos, node_size=100, node_color=node_colors, cmap=plt.cm.tab20b)
    nx.draw_networkx_labels(G, pos=label_pos, labels=labels, font_size=8, font_color='white', font_weight='800')

    plt.show()

In [ ]:
G = nx.Graph(np.array(population_a.weight))

a_check = False 
b_check = False 

while len(population_a.get_partition_points(0)) > 0 or len(population_b.get_partition_points(0)) > 0:
    population_a_excess = population_a.get_partition_points(0)
    population_b_excess = population_b.get_partition_points(0)
    
    count_null_population_a = 0
    for i in population_a_excess:
        partition_id, min_partition = -1, -1
        for j in [n for n in G.neighbors(i)]:
            partition = population_a.get_points().get(j).get('partition')
            if(min_partition == -1 and (partition != 0 and partition != max(temp_a_cross, key=temp_a_cross.get))):
                partition_id, min_partition = partition, len(population_a.get_partition_points(partition))
            elif(min_partition > len(population_a.get_partition_points(partition)) and (partition != 0 and partition != max(temp_a_cross, key=temp_a_cross.get))):
                partition_id, min_partition = partition, len(population_a.get_partition_points(partition))
        if(min_partition != -1):
            population_a.update_partition(i, partition_id)
        elif(min_partition == -1):
            count_null_population_a += 1
        elif(a_check):
            population_a.update_partition(i, max(temp_a_cross, key=temp_a_cross.get))
    if(count_null_population_a == len(population_a_excess)):
        a_check = True

    count_null_population_b = 0
    for i in population_b_excess:
        partition_id, min_partition = -1, -1
        for j in [n for n in G.neighbors(i)]:
            partition = population_b.get_points().get(j).get('partition')
            if(min_partition == -1 and (partition != 0 or partition != max(temp_b_cross, key=temp_b_cross.get))):
                partition_id, min_partition = partition, len(population_b.get_partition_points(partition))
            elif(min_partition > len(population_b.get_partition_points(partition)) and (partition != 0 and partition != max(temp_b_cross, key=temp_b_cross.get))):
                partition_id, min_partition = partition, len(population_b.get_partition_points(partition))

        if(min_partition != -1):
            population_b.update_partition(i, partition_id)
        elif(min_partition == -1):
            count_null_population_b += 1
        elif(b_check):
            population_b.update_partition(i, max(temp_b_cross, key=temp_b_cross.get))

    if(count_null_partition_b == len(population_b_excess)):
        b_check = True

In [ ]:
for population in [population_a, population_b]:
    population.mutate_graph()
    partition_result = population.get_partition()
    node_colors = [partition_result[node] for node in G.nodes()]
    labels = {node: f'{node}' for node in G.nodes()}
    label_pos = {k: (v[0], v[1]) for k, v in pos.items()}  

    plt.figure()
    nx.draw(G, pos, node_size=100, node_color=node_colors, cmap=plt.cm.tab20b)
    nx.draw_networkx_labels(G, pos=label_pos, labels=labels, font_size=8, font_color='white', font_weight='800')

    plt.show()